In [ ]:
import pandas as pd
import re 
import ast

In [ ]:
df = pd.read_csv('ohchr_SR-Cultural-Rights_2022_2024.csv')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df

In [ ]:
#check if there is more than one country in the country column 
df_country = df.copy()
country_split = df_country['country'].apply(lambda x: x.strip('[]').replace("'", "").split(', '))
has_multiple_countries = country_split.apply(lambda x: len(x) > 1)
df_country['has_multiple_countries'] = has_multiple_countries
df_country['has_multiple_countries'].value_counts()


In [ ]:
#spliting the country name to have it as a string 
country_df = df.copy()
splited_country = country_df['country'].apply(lambda x: x.strip('[]').replace("'", ""))
year_df = pd.DataFrame(splited_country.tolist(), index=country_df.index)
country_df = pd.concat([country_df, year_df], axis=1)
new_df = country_df.copy()
new_df = new_df.drop(columns='country')
new_df = new_df.rename(columns={0:'country'})
new_df.columns.values

In [ ]:
new_df.head(10)

In [ ]:
#clean the years
#df['Publishing_Year'] = df['Publishing_Year'].astype(int) / changes to ineger 

text = "CAT/C/KOR/CO/6 (CAT 2024 )	"
text = text.strip()
year_regex = r'\s\d{4}'
searching = re.findall(year_regex, text)
str_search = str(searching)
str_search.strip("[]").replace("'","").strip()


In [ ]:
#seperating year from code
new_df["Year"] = new_df["year"].apply(lambda x: re.findall(r"\s\d{4}", x.strip())[0] if re.findall(r"\s\d{4}", x.strip()) else None)
new_df["code"] = new_df["year"].apply(lambda x: re.findall(r"(.*?)\(", x.strip())[0] if re.findall(r"\((.*?)\)", x.strip()) else None)

new_df.head(10)
new_df = new_df.drop(columns='year')
new_df = new_df.rename(columns={'Year':'year'})
new_df.head(10)

In [ ]:
groups = new_df.copy()

splited_group = groups['concerned-group'].apply(lambda x: x.strip('[]').replace("'", ""))
group_df = pd.DataFrame(splited_group.tolist(), index=groups.index)
group_df = pd.concat([groups, group_df], axis=1)
group_df = group_df.drop(columns='concerned-group')
group_df = group_df.rename(columns={0:'concerned-group'})


In [ ]:
#spliting each group into a row so we have a good over view per group and case
df_concerened_groups = group_df.copy()
df_concerened_groups['concerned-group']=df_concerened_groups['concerned-group'].str.replace(r",\s+",' and ',regex=True)
df_concerened_groups['concerned-group']=df_concerened_groups['concerned-group'].str.split(' and ')
df_group_exploded = df_concerened_groups.explode('concerned-group')
df_group_exploded.head(10)


In [ ]:

themes = df_group_exploded.copy()

def safe_literal_eval(val):
    if isinstance(val, str):
        return ast.literal_eval(val)  # Converts the string representation to a Python list
    return val  # If already a list, do nothing

themes["human-rights-theme"] = themes["human-rights-theme"].apply(safe_literal_eval)

# Step 2: Explode the list directly (no regex needed)
df_exploded = themes.explode("human-rights-theme").reset_index(drop=True)

# Step 3: Explode the dataframe
df_exploded = themes.explode("human-rights-theme")
df_exploded.head(10)


In [ ]:
df_new_anno = df_exploded.copy()

anno_stiped = df_new_anno['annotaion type'].apply(lambda x: x.strip('[]').replace("'", ""))

anno_df_new = pd.DataFrame(anno_stiped.tolist(), index=df_new_anno.index)

anno_df_new = pd.concat([df_new_anno, anno_df_new], axis=1)

anno_df_new = anno_df_new.drop(columns='annotaion type')

anno_df_new = anno_df_new.rename(columns={0:'annotaion-type'})
anno_df_new.head(10)


In [ ]:
df_final = anno_df_new.copy()
df_final.shape
new_order = ["year","country","concerned-group", "human-rights-theme","annotaion-type","term", "code"]
df_final = df_final[new_order]


In [ ]:
df_index = df_final.copy()
df_index = df_index.reset_index(drop=True)  # `drop=True` removes the old index

df_index['country'].value_counts().unique()



In [ ]:
df_mech = df_index.copy()
df_mech["un-mechanism-code"] = df_mech["code"].apply(lambda x: re.findall(r"^.{3}", x.strip())[0] if re.findall(r"^.{3}", x.strip()) else None)
df_mech['un-mechanism'] = "Committee on the Rights of the Child"
df_mech = df_mech.drop(columns='code')
df_mech.head(10)

In [ ]:
new_order = ["un-mechanism-code","un-mechanism","year","country","concerned-group", "human-rights-theme","annotaion-type","term"]
df_mech = df_mech[new_order]
df_mech.head(10)

In [ ]:
final_df = df_mech.copy()
final_df.to_csv('Special-Rapporteur-in-the-field-of-cultural-rights.csv', index=False)

In [ ]:
final_df['country'].value_counts().head(10)

In [ ]:
final_df['human-rights-theme'].value_counts().head(10)

In [ ]:
final_df['concerned-group'].value_counts().head(10)

In [ ]:
final_df['term'].value_counts().head(10)